In [1]:
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', None)


In [22]:
df = pd.read_csv('data/movies.csv')
# df['poster']
# ordenar por ranking
df = df.sort_values(by='rank')
df.head(2)

,id,rank,title,fulltitle,year,image,crew,imdbrating,imdbratingcount,rated,released,runtime,genre,director,writer,actors,plot,language,country,awards,poster,metascore,imdbvotes,imdbid,type,dvd,boxoffice,production,website,response
18,tt0111161,1,The Shawshank Redemption,The Shawshank Redemption (1994),1994,https://m.media-amazon.com/images/M/MV5BMDFkYT...,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.3,2572473,R,14 Oct 1994,142 min,Drama,Frank Darabont,"Stephen King, Frank Darabont","Tim Robbins, Morgan Freeman, Bob Gunton",A banker convicted of uxoricide forms a friend...,English,United States,Nominated for 7 Oscars. 21 wins & 42 nominatio...,https://m.media-amazon.com/images/M/MV5BMDAyY2...,82,"3,059,994",tt0111161,movie,NaN,"$28,767,189",NaN,NaN,True
5,tt0068646,2,The Godfather,The Godfather (1972),1972,https://m.media-amazon.com/images/M/MV5BM2MyNj...,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.2,1771457,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","Marlon Brando, Al Pacino, James Caan",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",United States,Won 3 Oscars. 31 wins & 31 nominations total,https://m.media-amazon.com/images/M/MV5BNGEwYj...,100,"2,132,118",tt0068646,movie,NaN,"$136,381,073",NaN,NaN,True


In [8]:
def process_movie_data(df: pd.DataFrame) -> pd.DataFrame:
    # 1. Selecionar apenas as colunas desejadas e renomeá-las se necessário
    # Assumindo que os nomes das colunas de entrada são os mesmos que os desejados,
    # ou que você já fez uma renomeação prévia se forem diferentes.
    # Se os nomes forem diferentes na entrada, por favor, me informe.
    desired_columns = [
        'rank', 'title', 'year', 'imdbrating', 'imdbratingcount',
        'released', 'runtime', 'genre', 'director', 'language',
        'country', 'awards', 'metascore', 'imdbvotes', 'boxoffice'
    ]
    
    # Garantir que todas as colunas desejadas existam no DataFrame
    # Se alguma coluna estiver faltando, considere levantar um erro ou preenchê-la com NaN.
    # Por enquanto, vamos filtrar para garantir que só trabalhemos com o que temos.
    df = df[desired_columns].copy() # .copy() para evitar SettingWithCopyWarning

    # 2. Conversão de Tipos e Limpeza de Dados

    # rank: int
    df['rank'] = pd.to_numeric(df['rank'], errors='coerce').fillna(0).astype(int)

    # title: string (já deve ser string, apenas garantir)
    df['title'] = df['title'].astype(str)

    # year: int
    df['year'] = pd.to_numeric(df['year'], errors='coerce').fillna(0).astype(int)

    # imdbrating: int (assumindo que pode ser float e precisa ser int)
    df['imdbrating'] = pd.to_numeric(df['imdbrating'], errors='coerce').fillna(0).astype(int)

    # imdbratingcount: int (assumindo que pode ser string com vírgulas ou floats)
    # Remover vírgulas e converter para numérico
    df['imdbratingcount'] = df['imdbratingcount'].astype(str).str.replace(',', '', regex=False)
    df['imdbratingcount'] = pd.to_numeric(df['imdbratingcount'], errors='coerce').fillna(0).astype(int)

    # released: formato atualmente esta como 07 Jan 1947, colocar no formato MM/DD/YYYY
    # Função para parsear datas flexivelmente e formatar
    def format_date(date_str):
        if pd.isna(date_str) or not isinstance(date_str, str):
            return None
        try:
            # Tenta um formato comum (Day Month Year)
            return datetime.strptime(date_str, '%d %b %Y').strftime('%m/%d/%Y')
        except ValueError:
            try:
                # Tenta outro formato se o primeiro falhar
                return datetime.strptime(date_str, '%d %B %Y').strftime('%m/%d/%Y')
            except ValueError:
                # Se falhar, pode tentar outros formatos ou retornar None
                return None
    df['released'] = df['released'].apply(format_date)
    # Após a formatação, podemos deixá-lo como string ou converter para datetime,
    # mas o pedido foi para "MM/DD/YYYY" que é uma representação em string.
    # Se precisar do tipo datetime para análises futuras, seria assim:
    # df['released'] = pd.to_datetime(df['released'], format='%m/%d/%Y', errors='coerce')


    # runtime: ta como 130 min, deixar so o valor como int
    df['runtime'] = df['runtime'].astype(str).str.extract(r'(\d+)').astype(float).fillna(0).astype(int)

    # genre: lista de string, atualmente so esta separado por virgula
    # Manter como string, mas garantir que está limpo (pode ser usado para splitar mais tarde)
    df['genre'] = df['genre'].astype(str).str.strip() # Remove espaços extras nas pontas

    # director: string
    df['director'] = df['director'].astype(str)

    # language: string
    df['language'] = df['language'].astype(str)

    # country: string
    df['country'] = df['country'].astype(str)

    # awards: string
    df['awards'] = df['awards'].astype(str)

    # metascore: int (pode vir como string ou ter NaN)
    df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce').fillna(0).astype(int)

    # imdbvotes: int (pode vir como string com vírgulas)
    df['imdbvotes'] = df['imdbvotes'].astype(str).str.replace(',', '', regex=False)
    df['imdbvotes'] = pd.to_numeric(df['imdbvotes'], errors='coerce').fillna(0).astype(int)

    # boxoffice: atualemnte ta $1,483,643, deixar so o valor
    # Remover $, vírgulas e converter para numérico
    df['boxoffice'] = df['boxoffice'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
    df['boxoffice'] = pd.to_numeric(df['boxoffice'], errors='coerce').fillna(0).astype(int)
    
    return df

# Processar os dados
processed_df = process_movie_data(df)


In [83]:
# 1. 🎬 Top 10 IMDb Ratings
topN_rated_df = (
    df[[
        'rank', 'title', 'year', 'imdbrating', 'imdbratingcount',
        'released', 'runtime', 'genre', 'director', 'language',
        'country', 'awards', 'metascore', 'imdbvotes', 'boxoffice'
    ]]
    .dropna()
    .sort_values(by='imdbrating', ascending=False)
    .head(10)
)

# 2. 🎭 Distribuição por Gênero
genre_series = df['genre'].dropna().str.split(', ').explode()
movies_by_genre_df = genre_series.value_counts().rename_axis('genre').reset_index(name='count')

# 3. Movier per country
df['country'] = df['country'].fillna('').astype(str)
countries_exploded = df['country'].str.split(',', expand=True).stack().str.strip()
country_counts = countries_exploded.value_counts().reset_index()
country_counts.columns = ['Country', 'MovieCount']
country_counts = country_counts.sort_values(by='MovieCount', ascending=False)

# 4. 📆 Filmes por ano
movies_per_year_df = (
    df['year'].dropna().astype(int).value_counts().sort_index(ascending=False).rename_axis('year').reset_index(name='count')
)


# 5. 💵 Receita total por ano (Box Office)
df['boxoffice_clean'] = (
    df['boxoffice'].astype(str)
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)  
    .replace('nan', '0') 
    .astype(int)
)
box_office_df = (
    df.groupby('year', as_index=False)['boxoffice_clean']
    .sum()
    .rename(columns={'boxoffice_clean': 'total_box_office'})
    .sort_values(by='year', ascending=False)
)

# 6. Top 5 Diretores mais frequentes name x count movie
director_series = df['director'].dropna().str.split(', ').explode()
top_directors_df = (
    director_series.value_counts().head().rename_axis('director').reset_index(name='movie_count')
)

director_series

18          Frank Darabont
5     Francis Ford Coppola
31       Christopher Nolan
6     Francis Ford Coppola
2             Sidney Lumet
14        Steven Spielberg
26           Peter Jackson
17       Quentin Tarantino
22           Peter Jackson
4             Sergio Leone
15         Robert Zemeckis
25           David Fincher
33       Christopher Nolan
27           Peter Jackson
9           Irvin Kershner
24          Lana Wachowski
24         Lilly Wachowski
11         Martin Scorsese
7             Milos Forman
19           David Fincher
1           Akira Kurosawa
0              Frank Capra
12          Jonathan Demme
23        Steven Spielberg
30      Fernando Meirelles
30              Kátia Lund
20         Roberto Benigni
21          Frank Darabont
32       Christopher Nolan
8             George Lucas
13           James Cameron
10         Robert Zemeckis
28          Hayao Miyazaki
3         Alfred Hitchcock
29          Roman Polanski
16              Luc Besson
34            Bong Joon Ho
N

,director,movie_count
0,Christopher Nolan,3
1,Peter Jackson,3
2,Francis Ford Coppola,2
3,Frank Darabont,2
4,David Fincher,2
